In [107]:
import pandas as pd
import re, string, math
import numpy as np
import seaborn as sns
from sklearn.model_selection import train_test_split

import warnings
warnings.filterwarnings('ignore')

In [118]:
train_url="/home/ketan/Documents/Projects/learning/Saggy/Practice/prac2/train.csv"
test_url="/home/ketan/Documents/Projects/learning/Saggy/Practice/prac2/dev.csv"
train_data=pd.read_csv(train_url)
test_data=pd.read_csv(test_url)
#train_data, test_data=train_test_split(dataset, test_size=0.3, random_state=22)

In [119]:
train_data.head()

,Insult,Comment
0,1,"""You fuck your dad."""
1,0,"""i really don't understand your point.\xa0 It ..."
2,0,"""A\\xc2\\xa0majority of Canadians can and has ..."
3,0,"""listen if you dont wanna get married to a man..."
4,0,"""C\xe1c b\u1ea1n xu\u1ed1ng \u0111\u01b0\u1edd..."


In [120]:
test_data.head()
test_data.shape,train_data.shape

((2234, 2), (3947, 2))

In [121]:
reg = re.compile("([a-z ])", re.IGNORECASE | re.DOTALL | re.MULTILINE)
def tokenize(comment_text):
    comment_text = comment_text.lower()
    #print(comment_text)
    comment_text = comment_text.translate(comment_text.maketrans(string.punctuation,' ' * len(string.punctuation)))
    comment_text = "".join(reg.findall(comment_text))
    #print(comment_text)
    return comment_text.strip()
def populate_dict(x):
    #print(x)
    #print(list(filter(None, tokenize(x).split(" "))))
    vocab.update(list(filter(None,tokenize(x).split(" "))))    

In [130]:
from sklearn.feature_extraction.text import CountVectorizer
vocab=set()
cv = CountVectorizer(vocabulary=vocab)

np.vectorize(populate_dict)(train_data['Comment'])
print("Length of vocab is:",len(vocab))

Length of vocab is: 15914


In [126]:
from sklearn.feature_extraction.text import TfidfVectorizer
vocab=set()
cv = TfidfVectorizer(vocabulary=vocab,stop_words='english')

np.vectorize(populate_dict)(train_data['Comment'])
print("Length of vocab is:",len(vocab))

Length of vocab is: 15914


In [131]:
#train_data['Comments']=train_data.apply(lambda x: list(filter(None,tokenize(x['Comment']).split(' '))), axis=1)
#train_data['Comments']=train_data.apply(lambda x: list(filter(None,tokenize(x['Comment']))), axis=1)
# cv.fit_transform(
#     train_data.apply(lambda x: list(filter(None,tokenize(x['Comment']).split(' '))), axis=1)
# )
features=cv.fit_transform(
    train_data.apply(lambda x: tokenize(x['Comment']), axis=1)
).toarray()
features

array([[0, 0, 0, ..., 0, 0, 0],
       [0, 0, 0, ..., 0, 0, 0],
       [0, 0, 0, ..., 0, 0, 0],
       ...,
       [0, 0, 0, ..., 0, 0, 0],
       [0, 0, 0, ..., 0, 0, 0],
       [0, 0, 0, ..., 0, 0, 0]])

In [132]:
from sklearn.model_selection import train_test_split #for split the data
from sklearn.linear_model import LogisticRegression # Logistic Regression
from sklearn.metrics import accuracy_score  #for accuracy_score

from sklearn.model_selection import KFold #for K-fold cross validation
from sklearn.model_selection import cross_val_score
from sklearn.model_selection import cross_val_predict
from sklearn.metrics import confusion_matrix

lr_model=LogisticRegression(max_iter=1000)
lr_model.fit(features,train_data['Insult'])

LogisticRegression(C=1.0, class_weight=None, dual=False, fit_intercept=True,
          intercept_scaling=1, max_iter=1000, multi_class='warn',
          n_jobs=None, penalty='l2', random_state=None, solver='warn',
          tol=0.0001, verbose=0, warm_start=False)

In [133]:
X_test=cv.fit_transform(
    test_data.apply(lambda x: tokenize(x['Comment']), axis=1)
).toarray()
Y_test=test_data['Insult']
lr_prediction=lr_model.predict(X_test)
print("Accuracy is: ",round(accuracy_score(lr_prediction,Y_test)*100,2),"%")

Accuracy is:  70.55 %
